In [1]:
from PyQt5 import QtCore, QtGui, QtWidgets
from PyQt5.QtGui import QPixmap
import sys
from pytube import YouTube
import requests
import cv2

In [2]:
class Ui_ytVideoDwonloader(object):
    def sort_video(self,x):
        patterns = "144p 240p 360p 480p 720p 1080p 1440p 2160p".split(" ")
        for i in range(len(patterns)):
            pattern = patterns[i]
            if(x.find(pattern) == 0):
                return (i,x.replace(pattern,''))
        return (len(patterns)+1,x)
    def sort_audio(self,x):
        patterns = "50 70 128 160".split(" ")
        for i in range(len(patterns)):
            pattern = patterns[i]
            if(x.find(pattern) == 0):
                return (i,x.replace(pattern,''))
        return (len(patterns)+1,x)    
    def onSearchClicked(self):
        self.typeComboBox.clear()
        yt_link = self.ytLinkEdit.text()
        self.youtube_video = YouTube(yt_link)
        response = requests.get(self.youtube_video.thumbnail_url)
        file = open("sample_image.png", "wb")
        file.write(response.content)
        file.close()
        img = cv2.imread("sample_image.png")
        img = cv2.resize(img,(791,241))
        cv2.imwrite("sample_image.png",img)
        pixmap = QPixmap('sample_image.png')
        self.ytPicLabel.setPixmap(pixmap)
        types = []
        quality_video = []
        quality_audio = []
        for formats in self.youtube_video.streams:
            types.append(formats.mime_type[0:5])
            if("video" in formats.mime_type):
                quality_video.append(formats.resolution + f" {formats.fps}fps" + " " + formats.mime_type[6:])
            if("audio" in formats.mime_type):
                quality_audio.append(formats.abr)
        types = list(set(types))
        quality_video = list(set(quality_video))
        quality_audio = list(set(quality_audio))
        quality_audio = sorted(quality_audio,key=self.sort_audio)
        types.sort()
        quality_video = sorted(quality_video,key= self.sort_video)
        self.quality_video = quality_video
        self.quality_audio = quality_audio
        self.types = types
        self.typeComboBox.addItems(self.types)
        self.ytLinkEdit.setReadOnly(True)
        self.searchBtn.hide()
        self.resetBtn.show()
        self.downloadBtn.show()
        self.setQuality()
    def onDownloadClicked(self):
        quality = self.QualityComboBox.currentText().split(" ")
        for formats in self.youtube_video.streams:
            if(quality[-1] in formats.mime_type):
                if(quality[0] in formats.resolution):
                    if(int(quality[1][:-3]) == formats.fps):
                        formats.download()
                        return
    def onResetClicked(self):
        self.ytPicLabel.clear()
        self.ytLinkEdit.setReadOnly(False)
        self.ytLinkEdit.clear()
        self.searchBtn.show()
        self.resetBtn.hide()
        self.downloadBtn.hide()
        self.QualityComboBox.clear()
        self.typeComboBox.clear()
        self.youtube_video = None
        
    def setQuality(self):
        self.QualityComboBox.clear()
        if("video" in self.typeComboBox.currentText()):
            item = self.quality_video
        else:
            item = self.quality_audio
        self.QualityComboBox.addItems(item)
    def setupUi(self, ytVideoDwonloader):
        ytVideoDwonloader.setObjectName("ytVideoDwonloader")
        ytVideoDwonloader.resize(1040, 670)
        ytVideoDwonloader.setMinimumSize(QtCore.QSize(1040, 670))
        ytVideoDwonloader.setMaximumSize(QtCore.QSize(1040, 670))
        font = QtGui.QFont()
        font.setPointSize(8)
        ytVideoDwonloader.setFont(font)
        ytVideoDwonloader.setUnifiedTitleAndToolBarOnMac(False)
        self.centralwidget = QtWidgets.QWidget(ytVideoDwonloader)
        self.centralwidget.setObjectName("centralwidget")
        self.ytPicLabel = QtWidgets.QLabel(self.centralwidget)
        self.ytPicLabel.setGeometry(QtCore.QRect(120, 30, 791, 241))
        self.ytPicLabel.setText("")
        self.ytPicLabel.setObjectName("ytPicLabel")
        self.ytLinkEdit = QtWidgets.QLineEdit(self.centralwidget)
        self.ytLinkEdit.setGeometry(QtCore.QRect(210, 330, 641, 31))
        self.ytLinkEdit.setObjectName("ytLinkEdit")
        self.ytLinkEdit.editingFinished.connect(self.onSearchClicked)
        self.ytLinkText = QtWidgets.QLabel(self.centralwidget)
        self.ytLinkText.setGeometry(QtCore.QRect(110, 330, 101, 31))
        font = QtGui.QFont()
        font.setPointSize(8)
        self.ytLinkText.setFont(font)
        self.ytLinkText.setAlignment(QtCore.Qt.AlignCenter)
        self.ytLinkText.setObjectName("ytLinkText")
        self.typeComboBox = QtWidgets.QComboBox(self.centralwidget)
        self.typeComboBox.setGeometry(QtCore.QRect(210, 370, 641, 31))
        self.typeComboBox.setObjectName("typeComboBox")
        self.typeComboBox.activated[str].connect(self.setQuality)
        self.typeText = QtWidgets.QLabel(self.centralwidget)
        self.typeText.setGeometry(QtCore.QRect(110, 370, 101, 31))
        self.typeText.setAlignment(QtCore.Qt.AlignCenter)
        self.typeText.setObjectName("typeText")
        self.downloadBtn = QtWidgets.QPushButton(self.centralwidget)
        self.downloadBtn.setGeometry(QtCore.QRect(210, 460, 321, 41))
        self.downloadBtn.setObjectName("downloadBtn")
        self.downloadBtn.clicked.connect(self.onDownloadClicked)
        self.downloadBtn.hide()
        self.resetBtn = QtWidgets.QPushButton(self.centralwidget)
        self.resetBtn.setGeometry(QtCore.QRect(530, 460, 321, 41))
        self.resetBtn.setObjectName("resetBtn")
        self.resetBtn.clicked.connect(self.onResetClicked)
        self.resetBtn.hide()
        self.qualityText = QtWidgets.QLabel(self.centralwidget)
        self.qualityText.setGeometry(QtCore.QRect(110, 410, 101, 31))
        self.qualityText.setAlignment(QtCore.Qt.AlignCenter)
        self.qualityText.setObjectName("qualityText")
        self.QualityComboBox = QtWidgets.QComboBox(self.centralwidget)
        self.QualityComboBox.setGeometry(QtCore.QRect(210, 410, 641, 31))
        self.QualityComboBox.setObjectName("QualityComboBox")
        self.searchBtn = QtWidgets.QPushButton(self.centralwidget)
        self.searchBtn.setGeometry(QtCore.QRect(860, 330, 101, 31))
        self.searchBtn.setObjectName("searchBtn")
        self.searchBtn.clicked.connect(self.onSearchClicked)
        ytVideoDwonloader.setCentralWidget(self.centralwidget)

        self.retranslateUi(ytVideoDwonloader)
        QtCore.QMetaObject.connectSlotsByName(ytVideoDwonloader)

    def retranslateUi(self, ytVideoDwonloader):
        _translate = QtCore.QCoreApplication.translate
        ytVideoDwonloader.setWindowTitle(_translate("ytVideoDwonloader", "Youtube Video Downloader"))
        self.ytLinkText.setText(_translate("ytVideoDwonloader", "Enter Link"))
        self.typeText.setText(_translate("ytVideoDwonloader", "Choose Type"))
        self.downloadBtn.setText(_translate("ytVideoDwonloader", "Download"))
        self.resetBtn.setText(_translate("ytVideoDwonloader", "Reset"))
        self.qualityText.setText(_translate("ytVideoDwonloader", "Choose Quality"))
        self.searchBtn.setText(_translate("ytVideoDwonloader", "Search"))



In [3]:
app = QtWidgets.QApplication(sys.argv)
ytVideoDwonloader = QtWidgets.QMainWindow()
ui = Ui_ytVideoDwonloader()
ui.setupUi(ytVideoDwonloader)
ytVideoDwonloader.show()
sys.exit(app.exec_())

SystemExit: 0

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3426: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
